In [1]:
# Load pickled data
import pickle

# TODO: fill this in based on where you saved the training and testing data
training_file = 'train.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [2]:
import numpy as np

### To start off let's do a basic data summary.

# TODO: number of training examples
n_train = X_train.shape[0]

# TODO: number of testing examples
n_test = X_test.shape[0]

# TODO: what's the shape of an image?
image_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])

# TODO: how many classes are in the dataset
n_classes = np.unique(y_train).shape[0]

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 39209
Number of testing examples = 12630
Image data shape = (32, 32, 3)
Number of classes = 43


In [29]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import KFold
import tensorflow as tf
%matplotlib inline

/home/tunguz/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
np.max(X_test)

255

In [10]:
### Generate data additional (if you want to!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.

X_train_scaled = (X_train - np.mean(X_train))/(256-np.mean(X_train))
X_test_scaled = (X_test- np.mean(X_train))/(256-np.mean(X_train))

In [16]:
np.mean(X_test_scaled)

-0.0029803106171309626

In [18]:
ohe = OneHotEncoder(sparse=False)

In [19]:
ohe.fit(y_train.reshape(-1,1))

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=False)

In [20]:
y_train = ohe.transform(y_train.reshape(-1,1))

In [21]:
y_test = ohe.transform(y_test.reshape(-1,1))

In [23]:
y_train

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [30]:
kf = KFold(n_train, n_folds=306, shuffle=True)

In [26]:
learning_rate = 0.003
batch_size = 128
training_epochs = 30

n_input = 32*32*3  # MNIST data input (Shape: 28*28)
n_classes = 43  # MNIST total classes (0-9 digits)

layer_width = {
    'layer_1': 64,
    'layer_2': 128,
    'layer_3': 256,
    'fully_connected_1': 1024,
    'fully_connected_2': 512
}

def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

# Store layers weight & bias
weights = {
    'layer_1': tf.Variable(tf.truncated_normal(
        [5, 5, 3, layer_width['layer_1']],stddev = 0.05)),
    'layer_2': tf.Variable(tf.truncated_normal(
        [5, 5, layer_width['layer_1'], layer_width['layer_2']], stddev = 0.05)),
    'layer_3': tf.Variable(tf.truncated_normal(
        [5, 5, layer_width['layer_2'], layer_width['layer_3']], stddev = 0.05)),
    'fully_connected_1': tf.Variable(tf.truncated_normal(
        [2048, layer_width['fully_connected_1']], stddev = 0.05)),
    'fully_connected_2': tf.Variable(tf.truncated_normal(
        [layer_width['fully_connected_1'], layer_width['fully_connected_2']], stddev = 0.05)),
    'out': tf.Variable(tf.truncated_normal(
        [layer_width['fully_connected_2'], n_classes], stddev = 0.05))
}
biases = {
    'layer_1': tf.Variable(tf.zeros(layer_width['layer_1'])),
    'layer_2': tf.Variable(tf.zeros(layer_width['layer_2'])),
    'layer_3': tf.Variable(tf.zeros(layer_width['layer_3'])),
    'fully_connected_1': tf.Variable(tf.zeros(layer_width['fully_connected_1'])),
    'fully_connected_2': tf.Variable(tf.zeros(layer_width['fully_connected_2'])),
    'out': tf.Variable(tf.zeros(n_classes))
}

# Create model
def conv_net(x, weights, biases):
    # Layer 1
    conv1 = conv2d(x, weights['layer_1'], biases['layer_1'])
    conv1 = maxpool2d(conv1)

    # Layer 2
    conv2 = conv2d(conv1, weights['layer_2'], biases['layer_2'])
    conv2 = maxpool2d(conv2)

    # Layer 3
    conv3 = conv2d(conv2, weights['layer_3'], biases['layer_3'])
    conv3 = maxpool2d(conv2)

    # Fully connected layer
    # Reshape conv3 output to fit fully connected layer input
    fc1 = tf.reshape(
        conv3,
        [-1, weights['fully_connected_1'].get_shape().as_list()[0]])
    fc1 = tf.add(
        tf.matmul(fc1, weights['fully_connected_1']),
        biases['fully_connected_1'])
    fc1 = tf.nn.relu(fc1)
    fc2 = tf.add(
        tf.matmul(fc1, weights['fully_connected_2']),
        biases['fully_connected_2'])
    fc2 = tf.nn.relu(fc2)

    # Output Layer - class prediction
    out = tf.add(tf.matmul(fc2, weights['out']), biases['out'])
    return out

# tf Graph input
x = tf.placeholder("float", [None, 32, 32, 3])
y = tf.placeholder("float", [None, n_classes])

logits = conv_net(x, weights, biases)

In [28]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    # Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))


# Initializing the variables
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [34]:
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        for (i, j) in kf:
            batch_x, batch_y = X_train_scaled[j], y_train[j]
            # Run optimization op (backprop) and cost op (to get loss value)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        # Display logs per epoch step
        c = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
        a = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
        print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c), "accuracy=", "{:.9f}".format(a))
    print("Optimization Finished!")


Epoch: 0001 cost= 3.298856735 accuracy= 0.148437500
Epoch: 0002 cost= 3.069041729 accuracy= 0.218750000
Epoch: 0003 cost= 2.766264677 accuracy= 0.296875000
Epoch: 0004 cost= 2.397610903 accuracy= 0.375000000
Epoch: 0005 cost= 2.029237747 accuracy= 0.460937500
Epoch: 0006 cost= 1.707121253 accuracy= 0.593750000
Epoch: 0007 cost= 1.423470020 accuracy= 0.664062500
Epoch: 0008 cost= 1.189630985 accuracy= 0.734375000
Epoch: 0009 cost= 1.002859235 accuracy= 0.757812500
Epoch: 0010 cost= 0.852782786 accuracy= 0.789062500
Epoch: 0011 cost= 0.735143185 accuracy= 0.812500000
Epoch: 0012 cost= 0.641408861 accuracy= 0.843750000
Epoch: 0013 cost= 0.562160790 accuracy= 0.859375000
Epoch: 0014 cost= 0.494858384 accuracy= 0.890625000
Epoch: 0015 cost= 0.438961297 accuracy= 0.914062500
Epoch: 0016 cost= 0.391159326 accuracy= 0.921875000
Epoch: 0017 cost= 0.351194859 accuracy= 0.937500000
Epoch: 0018 cost= 0.317304254 accuracy= 0.937500000
Epoch: 0019 cost= 0.286753058 accuracy= 0.945312500
Epoch: 0020 

In [36]:
with tf.Session() as sess:
    sess.run(init)
    print(
            "Accuracy:",
            accuracy.eval({x: X_test_scaled, y: y_test}))

Accuracy: 0.0266825
